In [5]:
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib
import zipfile
import os
import pandas as pd
import warnings
import rarfile, csv
import torch.nn as nn
import torch
import torch.optim as optim
import torch.utils.data as data_utils
from scipy.stats import pearsonr
import seaborn as sns
from sklearn.model_selection import train_test_split
from random import shuffle
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import random
import torch.nn.functional as F
import sklearn
import skorch
import pickle
import math
from collections import Counter
from fastprogress import master_bar, progress_bar




from skorch.utils import params_for
from torch.autograd import Variable




warnings.filterwarnings(action='once')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


%matplotlib inline

### RNN classifier


In [24]:
import time
import math

epoch_num = 20
learning_rate = 0.00001
dropout_p = 0
enc_hidden_size = 128
dec_hidden_size = 128
n_features = len(dataset_encoded_normalized.columns)-1
LSTM_layer = 3
LSTM_hidden = 512
use_cuda = True
BATCH_SIZE = 1
L2 = 0.001

In [122]:
class RNN(nn.Module):

    def __init__(self, input_size, enc_hidden_size, dec_hidden_size, LSTM_hidden, LSTM_layer, output_size, dropout_p):

        super(RNN, self).__init__()

        self.enc_hidden_size = enc_hidden_size
        
        self.dec_hidden_size = dec_hidden_size

        self.LSTM_hidden_size = LSTM_hidden

        self.enc = nn.Linear(input_size , enc_hidden_size).cuda()
        self.relu_enc = nn.ReLU().cuda()
        
        self.enc2 = nn.Linear(enc_hidden_size , 256).cuda()
        self.relu_enc2 = nn.ReLU().cuda()

        #self.lstm = nn.LSTM(input_size = enc_hidden_size, hidden_size = LSTM_hidden, num_layers = LSTM_layer,
        #                    dropout = dropout_p, bidirectional = True).cuda()
        
        self.lstm = nn.LSTM(input_size = 256, hidden_size = LSTM_hidden, num_layers = LSTM_layer,
                            dropout = dropout_p, bidirectional = True).cuda()

        self.dec2 = nn.Linear( LSTM_hidden * 2 , 256).cuda()
        self.relu_dec2 = nn.ReLU().cuda()
        
        self.dec = nn.Linear(256 , dec_hidden_size).cuda()
        self.relu_dec = nn.ReLU().cuda()
                
        self.output = nn.Linear(dec_hidden_size , output_size).cuda()

        self.dropout = nn.Dropout(p = dropout_p).cuda()

        self.softmax = nn.LogSoftmax(dim=2).cuda()


    def one_step_forward(self, input, hidden):

        encoded = self.relu_enc2(self.enc2(self.relu_enc(self.enc(input))))
        
        lstm_out, hidden_t1 = self.lstm(encoded.unsqueeze(0), hidden)
        
        dec = self.relu_dec(self.dec(self.relu_dec2(self.dec2(lstm_out))))

        if self.training:
            dec = self.dropout(dec)

        output = self.output(dec)
        softmax = self.softmax(output)

        return softmax, hidden_t1
        

    def initHidden(self, batch_size, num_lstm_layers = 1):
        return (torch.zeros(2 * num_lstm_layers, batch_size, self.LSTM_hidden_size).to(device=device), torch.zeros(2 * num_lstm_layers, batch_size, self.LSTM_hidden_size).to(device=device))


In [126]:
class Trainer:
    
    def __init__(self, optimizer = torch.optim.Adam, learning_rate = learning_rate, L2 = L2,
                                                    dec_hidden_size = dec_hidden_size,
                                                    enc_hidden_size = enc_hidden_size,
                                                    LSTM_hidden = LSTM_hidden,
                                                    LSTM_layer = LSTM_layer,
                                                    n_features = n_features,
                                                    output_size = output_size,
                                                    dropout_p = dropout_p,
                                                    criterion = nn.NLLLoss()):
        
        self.module = RNN(input_size = n_features, dec_hidden_size = dec_hidden_size, enc_hidden_size = enc_hidden_size,
          LSTM_hidden = LSTM_hidden, LSTM_layer = LSTM_layer, output_size = output_size, dropout_p = dropout_p)
        self.module = self.module.to(device)
        self.LSTM_layer = LSTM_layer
        self.n_features = n_features
        self.criterion = criterion
        
        self.optimizer_adam = optimizer(self.module.parameters(), lr = learning_rate, weight_decay = L2)
        
        
    def train_step(self, Xi, yi):

        loss = None
        accuracy = None
        
        Xi = Xi.reshape(1,-1,self.n_features)
        hid = self.module.initHidden(batch_size = 1, num_lstm_layers = self.LSTM_layer)
        self.module.zero_grad()
        for i in range(Xi.shape[1]):
            
            output, hid = self.module.one_step_forward(torch.tensor(Xi[:,i,:]).type(torch.FloatTensor).to(device=device), hid)        
            
        probabilities = torch.exp(output)
        _, predicted = torch.max(output.data, 2)
        
        if not(yi is None):
            
            correct = (predicted.squeeze(0) == yi.type(torch.LongTensor).to(device=device)).sum()
            accuracy = correct
            loss = self.get_loss(output, yi)
        
            if self.module.training:

                loss.backward()
                self.optimizer_adam.step()

        return {'loss':  loss, 'probabilities': probabilities, 'acc': accuracy}

    def infer(self, Xi, yi=None):

        self.module.eval()
        output = self.train_step(Xi, yi)
        self.module.train()
        
        return output
    
    def get_loss(self, ys_pred, y_true):
        
        ys_pred = ys_pred.squeeze(0)
        y_true = y_true.squeeze(0)
       
        return self.criterion(ys_pred, y_true.type(torch.LongTensor).to(device = device))

In [116]:
fault_predictor = Trainer()
mb = master_bar(range(1, epoch_num))
mb2 = master_bar(range(1, epoch_num))
mb.names = ['Training Error', 'Validation Error']
mb2.names = ['Training acc', 'Validation acc']

epoch_tra_acc = {}
epoch_tra_error = {}

epoch_evl_acc = {}
epoch_evl_error = {}

In [ ]:
for i in mb:
    

    tra_acc = []
    tra_error = []
    
    evl_acc = []
    evl_error = []
    
    
    for j in progress_bar(range(len(X_tra[0:1000])), parent=mb):
        
        mb.child.comment = f'training'
        
        stat = fault_predictor.train_step(X_tra[j], y_tra[j])
        tra_acc.append(stat['acc'].cpu().detach().numpy())
        tra_error.append(stat['loss'].cpu().detach().numpy())
        
    mb.write("### Training ### Epoch:{} loss:{} acc:{}".format(i , round(np.sum(tra_error)/(len(tra_error)*1.), 5), round(np.sum(tra_acc)/(len(tra_acc)*1.), 5)))
    epoch_tra_error[i] = round(np.sum(tra_error)/(len(tra_error)*1.), 5)
    epoch_tra_acc[i] = round(np.sum(tra_acc)/(len(tra_acc)*1.), 5)
    #graphs_acc = [[epoch_tra_error.keys(), epoch_tra_error.values()],[epoch_evl_error.keys(), epoch_evl_error.values()]]
    graphs_acc = [[epoch_tra_error.keys(), epoch_tra_error.values()],[epoch_evl_error.keys(), epoch_evl_error.values()]]
    graphs_acc2 = [[epoch_tra_acc.keys(), epoch_tra_acc.values()],[epoch_evl_acc.keys(), epoch_evl_acc.values()]]
    mb.update_graph(graphs_acc)
    mb2.update_graph(graphs_acc2)
    
    if (i % 1) == 0:

        for k in progress_bar(range(len(X_test)), parent=mb):
            
            mb.child.comment = f'validation'
            
            stat = fault_predictor.infer(X_test[k], y_test[k])
            evl_acc.append(stat['acc'].cpu().detach().numpy()) 
            evl_error.append(stat['loss'].cpu().detach().numpy())
        mb.write("### Validation ### Epoch:{} loss:{} acc:{}".format(i , round(np.sum(evl_error)/(len(evl_error)*1.), 5), round(np.sum(evl_acc)/(len(evl_acc)*1.), 5)))
        
        epoch_evl_acc[i] = round(np.sum(evl_acc)/(len(evl_acc)*1.), 5)
        epoch_evl_error[i] = round(np.sum(evl_error)/(len(evl_error)*1.), 5)
        
        #graphs_acc = [[epoch_tra_error.keys(), epoch_tra_error.values()], [epoch_evl_error.keys(), epoch_evl_error.values()]]
        graphs_acc = [[epoch_tra_error.keys(), epoch_tra_error.values()],[epoch_evl_error.keys(), epoch_evl_error.values()]]
        graphs_acc2 = [[epoch_tra_acc.keys(), epoch_tra_acc.values()],[epoch_evl_acc.keys(), epoch_evl_acc.values()]]
        mb.update_graph(graphs_acc)
        mb2.update_graph(graphs_acc2)

In [164]:
predd = []

for i in range(len(X_test)):
    pred = fault_predictor.infer(X_test[i])
    _, predicted = torch.max(pred['probabilities'].squeeze(0), 1)
    predd.append(predicted)

c:\users\administrator\pytorch\lib\site-packages\ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
cr = classification_report(y_test.astype(int), [predd[i].cpu().detach().numpy() for i in range(len(predd))])
print(cr)

In [166]:
def vis_conf_mat(conf_arr):
    norm_conf = []
    for i in conf_arr:
        a = 0
        tmp_arr = []
        a = sum(i, 0)
        for j in i:
            tmp_arr.append(float(j)/float(a))
        norm_conf.append(tmp_arr)

    fig = plt.figure(figsize=(8, 6))
    plt.clf()
    ax = fig.add_subplot(111)
    ax.set_aspect(1)
    res = ax.imshow(np.array(norm_conf), cmap=plt.cm.jet, 
                    interpolation='nearest')

    width, height = conf_arr.shape

    for x in range(width):
        for y in range(height):
            ax.annotate(str(conf_arr[x][y]), xy=(y, x), 
                        horizontalalignment='center',
                        verticalalignment='center')

    cb = fig.colorbar(res)
    alphabet = ['RAN:SITE DOWN (2G_3G_4G)', 'RAN:EXT/MAIN FAIL', 'RAN:EXT/DOOR OPEN', 'RAN:EXT/HIGH TEMP', 'RAN:EXT/RECTIFIRE FAIL', 'RAN:EXT/BATTERY_ALARM', 'RAN:PE', 'RAN:CELL DOWN(2G_3G_4G)', 'RAN:EXT/BATTERY_DISCONNECT']
    plt.xticks(range(width), alphabet[:width],rotation='vertical')
    plt.yticks(range(height), alphabet[:height])
    plt.savefig('confusion_matrix.png', format='png')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  


conf_arr = confusion_matrix(y_test.astype(int), [predd[i].cpu().detach().numpy() for i in range(len(predd))])

vis_conf_mat(conf_arr)

In [37]:
torch.save(fault_predictor, './model/RNN2')